In [1]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import random

**Improvements**

- Name Entity Recognition to remove those
- PoS, only look at e.g. adjectives?

# Crunch
Only run when adding new stuff to cleaner, otherwise use the CSV in the next section

In [2]:
# reading original
original = pd.read_csv('labeledTrainData.tsv', sep="\t")
original.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [3]:
stops = stopwords.words("english")
porter = PorterStemmer()

def clean_text(text):
    words = word_tokenize(text)
    words_no_punc = [word.lower() for word in words if word.isalpha()]
    no_stop = [word for word in words_no_punc if word not in stops]
    stems = [porter.stem(word) for word in no_stop]
    clean = ' '.join(stems)

    return clean

original["review_clean"] = original["review"].apply(lambda text: clean_text(text))
original.to_csv('train_clean.csv', index=False)

# Exploration

In [10]:
df = pd.read_csv('train_clean.csv', usecols=["sentiment", "review", "review_clean"])
df.head()

,sentiment,review,review_clean
0,1,With all this stuff going down at the moment w...,stuff go moment mj start listen music watch od...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war timothi hine entertain film obviou...
2,0,The film starts with a manager (Nicholas Bell)...,film start manag nichola bell give welcom inve...
3,0,It must be assumed that those who praised this...,must assum prais film greatest film opera ever...
4,1,Superbly trashy and wondrously unpretentious 8...,superbl trashi wondrous unpretenti exploit hoo...


In [11]:
random_nr = random.randint(0, len(df))

old = df["review"][random_nr]
new = df["review_clean"][random_nr]

print(f'Sentiment: {df["sentiment"][random_nr]}')
print(f'\nBefore ({len(old)} chars):')
print(old)
print(f'\nAfter ({len(new)} chars):')
print(new)

Sentiment: 1

Before (813 chars):
An excellent film depicting the cross currents in the lives of a multi-ethnic mix of not so ordinary people in the rural Pacific Northwest. Solid directing and writing along with fine acting, especially the performances by Kwami Taha and Dan Stowe. Interestingly, this film was made in the same year as the highly successful \Crash,\" written and directed by Paul Haggis. The pace of the action may not be as frantic as that in urban Los Angeles, and the characters may seem to be better acquainted with each other in \"Apart From That,\" but the personal relationships of the characters are as flawed and troubled and their stories as resonant as any of those in \"Crash.\" For those viewers who appreciated \"Crash\" this is a must see film. Also, fans of Jim Jarmusch and John Cassavetes will like this movie."

After (421 chars):
excel film depict cross current live mix ordinari peopl rural pacif northwest solid direct write along fine act especi perform kwami